# Reading Data - JSON Files

**Technical Accomplishments:**
- Read data from:
  * JSON without a Schema
  * JSON with a Schema

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Created user-specific database

Using the database arpit_mittal_team_telstra_com_db .

All done!

In [0]:
%run "./Includes/Utility-Methods"

Declared various utility methods: 
 Declared printRecordsPerPartition( df:DataFrame ) for diagnostics 
 Declared computeFileStats( path:String ) returns (count:Long, bytes:Long) for diagnostics 
 Declared tracker for benchmarking 
 Declared cacheAs( df:DataFrame, name:String, level:StorageLevel ) for better debugging 
 Declared benchmarkCount( lambda:DataFrame ) returns (df:DataFrame, total:Long, duration:Long) for diagnostics 
 Declared untilStreamIsReady( name:String ) to control workflow 
 
 All done!

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Reading from JSON w/ InferSchema

Reading in JSON isn't that much different than reading in CSV files.

Let's start with taking a look at all the different options that go along with reading in JSON files.

### JSON Lines

Much like the CSV reader, the JSON reader also assumes...
* That there is one JSON object per line and...
* That it's delineated by a new-line.

This format is referred to as **JSON Lines** or **newline-delimited JSON** 

More information about this format can be found at <a href="http://jsonlines.org/" target="_blank">http://jsonlines.org</a>.

** *Note:* ** *Spark 2.2 was released on July 11th 2016. With that comes File IO improvements for CSV & JSON, but more importantly, **Support for parsing multi-line JSON and CSV files**. You can read more about that (and other features in Spark 2.2) in the <a href="https://databricks.com/blog/2017/07/11/introducing-apache-spark-2-2.html" target="_blank">Databricks Blog</a>.*

### The Data Source
* For this exercise, we will be using the file called **snapshot-2016-05-26.json** (<a href="https://wikitech.wikimedia.org/wiki/Stream.wikimedia.org/rc" target="_blank">4 MB</a> file from Wikipedia).
* The data represents a set of edits to Wikipedia articles captured in May of 2016.
* It's located on the DBFS at **dbfs:/mnt/training/wikipedia/edits/snapshot-2016-05-26.json**
* Like we did with the CSV file, we can use **&percnt;fs ls ...** to view the file on the DBFS.

In [0]:
%fs ls dbfs:/mnt/training/wikipedia/edits/snapshot-2016-05-26.json

path,name,size
dbfs:/mnt/training/wikipedia/edits/snapshot-2016-05-26.json,snapshot-2016-05-26.json,4222431


Like we did with the CSV file, we can use **&percnt;fs head ...** to peek at the first couple lines of the JSON file.

In [0]:
%fs head dbfs:/mnt/training/wikipedia/edits/snapshot-2016-05-26.json

[Truncated to first 65536 bytes]
{"isRobot":false,"channel":"#pl.wikipedia","timestamp":"2016-05-26T19:19:14.367Z","url":"https://pl.wikipedia.org/w/index.php?diff=45897271&oldid=42573382","isUnpatrolled":false,"page":"Banda bat´ki Knysza","wikipedia":"pl","wikipediaURL":"http://pl.wikipedia.org","comment":"plik, drobne techniczne, kat.","userURL":"http://pl.wikipedia.org/wiki/User:Czymjestlogika","pageURL":"http://pl.wikipedia.org/wiki/Banda_bat´ki_Knysza","delta":116,"flag":"","isNewPage":false,"isAnonymous":false,"geocoding":{"countryCode2":null,"city":null,"latitude":null,"country":null,"longitude":null,"stateProvince":null,"countryCode3":null},"user":"Czymjestlogika","namespace":"article"}
{"isRobot":false,"channel":"#fr.wikipedia","timestamp":"2016-05-26T19:19:14.515Z","url":"https://fr.wikipedia.org/w/index.php?diff=126531898&oldid=118306121&rcid=188792265","isUnpatrolled":true,"page":"La Méprise (roman)","wikipedia":"fr","wikipediaURL":"http://fr.wikipedia.org","comment":"/* Résumé */","userURL":"http://fr.wikipedia.org/wiki/User:Polipille","pageURL":"http://fr.wikipedia.org/wiki/La_Méprise_(roman)","delta":-1,"flag":"!M","isNewPage":false,"isAnonymous":false,"geocoding":{"countryCode2":null,"city":null,"latitude":null,"country":null,"longitude":null,"stateProvince":null,"countryCode3":null},"user":"Polipille","namespace":"article"}
{"isRobot":true,"channel":"#en.wikipedia","timestamp":"2016-05-26T19:19:14.586Z","url":"https://en.wikipedia.org/w/index.php?diff=722229003&oldid=720692086","isUnpatrolled":false,"page":"Carlos Zeballos","wikipedia":"en","wikipediaURL":"http://en.wikipedia.org","comment":"Rescuing 1 sources. #IABot","userURL":"http://en.wikipedia.org/wiki/User:Cyberbot II","pageURL":"http://en.wikipedia.org/wiki/Carlos_Zeballos","delta":110,"flag":"B","isNewPage":false,"isAnonymous":false,"geocoding":{"countryCode2":null,"city":null,"latitude":null,"country":null,"longitude":null,"stateProvince":null,"countryCode3":null},"user":"Cyberbot II","namespace":"article"}
{"isRobot":true,"channel":"#sv.wikipedia","timestamp":"2016-05-26T19:19:14.739Z","url":"https://sv.wikipedia.org/w/index.php?oldid=35100063&rcid=81393046","isUnpatrolled":false,"page":"Doliana","wikipedia":"sv","wikipediaURL":"http://sv.wikipedia.org","comment":"Botskapande Grekland omdirigering","userURL":"http://sv.wikipedia.org/wiki/User:Lsjbot","pageURL":"http://sv.wikipedia.org/wiki/Doliana","delta":26,"flag":"NB","isNewPage":true,"isAnonymous":false,"geocoding":{"countryCode2":null,"city":null,"latitude":null,"country":null,"longitude":null,"stateProvince":null,"countryCode3":null},"user":"Lsjbot","namespace":"article"}
{"isRobot":true,"channel":"#en.wikipedia","timestamp":"2016-05-26T19:19:15.249Z","url":"https://en.wikipedia.org/w/index.php?diff=722229004&oldid=720456554","isUnpatrolled":false,"page":"Talk:Carlos Zeballos","wikipedia":"en","wikipediaURL":"http://en.wikipedia.org","comment":"Notification of altered sources needing review #IABot","userURL":"http://en.wikipedia.org/wiki/User:Cyberbot II","pageURL":"http://en.wikipedia.org/wiki/Talk:Carlos_Zeballos","delta":1063,"flag":"B","isNewPage":false,"isAnonymous":false,"geocoding":{"countryCode2":null,"city":null,"latitude":null,"country":null,"longitude":null,"stateProvince":null,"countryCode3":null},"user":"Cyberbot II","namespace":"talk"}
{"isRobot":false,"channel":"#de.wikipedia","timestamp":"2016-05-26T19:19:15.309Z","url":"https://de.wikipedia.org/w/index.php?diff=154727294&oldid=154727191","isUnpatrolled":false,"page":"Avantasia","wikipedia":"de","wikipediaURL":"http://de.wikipedia.org","comment":"/* Gesang */ ...auch Amanda war nur auf der einer Sonderedition des Albums beiliegenden Bonus-Live-CD vertreten.","userURL":"http://de.wikipedia.org/wiki/User:Johnny.m76","pageURL":"http://de.wikipedia.org/wiki/Avantasia","delta":-7,"flag":"","isNewPage":false,"isAnonymous":false,"geocoding":{"countryCode2":null,"city":null,"latitude":null,"country":null,"longitude":null,"stateProvince

### Read The JSON File

The command to read in JSON looks very similar to that of CSV.

In addition to reading the JSON file, we will also print the resulting schema.

In [0]:
jsonFile = "dbfs:/mnt/training/wikipedia/edits/snapshot-2016-05-26.json"

wikiEditsDF = (spark.read           # The DataFrameReader
    .option("inferSchema", "true")  # Automatically infer data types & column names
    .json(jsonFile)                 # Creates a DataFrame from JSON after reading in the file
 )
wikiEditsDF.printSchema()

root
-- channel: string (nullable = true)
-- comment: string (nullable = true)
-- delta: long (nullable = true)
-- flag: string (nullable = true)
-- geocoding: struct (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- countryCode2: string (nullable = true)
 |-- countryCode3: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stateProvince: string (nullable = true)
-- isAnonymous: boolean (nullable = true)
-- isNewPage: boolean (nullable = true)
-- isRobot: boolean (nullable = true)
-- isUnpatrolled: boolean (nullable = true)
-- namespace: string (nullable = true)
-- page: string (nullable = true)
-- pageURL: string (nullable = true)
-- timestamp: string (nullable = true)
-- url: string (nullable = true)
-- user: string (nullable = true)
-- userURL: string (nullable = true)
-- wikipedia: string (nullable = true)
-- wikipediaURL: string (nullable = true)

With our DataFrame created, we can now take a peak at the data.

But to demonstrate a unique aspect of JSON data (or any data with embedded fields), we will first create a temporary view and then view the data via SQL:

In [0]:
# create a view called wiki_edits
wikiEditsDF.createOrReplaceTempView("wiki_edits")

And now we can take a peak at the data with simple SQL SELECT statement:

In [0]:
%sql

SELECT * FROM wiki_edits 

channel,comment,delta,flag,geocoding,isAnonymous,isNewPage,isRobot,isUnpatrolled,namespace,page,pageURL,timestamp,url,user,userURL,wikipedia,wikipediaURL
#pl.wikipedia,"plik, drobne techniczne, kat.",116,,"List(null, null, null, null, null, null, null)",false,false,false,false,article,Banda bat´ki Knysza,http://pl.wikipedia.org/wiki/Banda_bat´ki_Knysza,2016-05-26T19:19:14.367Z,https://pl.wikipedia.org/w/index.php?diff=45897271&oldid=42573382,Czymjestlogika,http://pl.wikipedia.org/wiki/User:Czymjestlogika,pl,http://pl.wikipedia.org
#fr.wikipedia,/* Résumé */,-1,!M,"List(null, null, null, null, null, null, null)",false,false,false,true,article,La Méprise (roman),http://fr.wikipedia.org/wiki/La_Méprise_(roman),2016-05-26T19:19:14.515Z,https://fr.wikipedia.org/w/index.php?diff=126531898&oldid=118306121&rcid=188792265,Polipille,http://fr.wikipedia.org/wiki/User:Polipille,fr,http://fr.wikipedia.org
#en.wikipedia,Rescuing 1 sources. #IABot,110,B,"List(null, null, null, null, null, null, null)",false,false,true,false,article,Carlos Zeballos,http://en.wikipedia.org/wiki/Carlos_Zeballos,2016-05-26T19:19:14.586Z,https://en.wikipedia.org/w/index.php?diff=722229003&oldid=720692086,Cyberbot II,http://en.wikipedia.org/wiki/User:Cyberbot II,en,http://en.wikipedia.org
#sv.wikipedia,Botskapande Grekland omdirigering,26,NB,"List(null, null, null, null, null, null, null)",false,true,true,false,article,Doliana,http://sv.wikipedia.org/wiki/Doliana,2016-05-26T19:19:14.739Z,https://sv.wikipedia.org/w/index.php?oldid=35100063&rcid=81393046,Lsjbot,http://sv.wikipedia.org/wiki/User:Lsjbot,sv,http://sv.wikipedia.org
#en.wikipedia,Notification of altered sources needing review #IABot,1063,B,"List(null, null, null, null, null, null, null)",false,false,true,false,talk,Talk:Carlos Zeballos,http://en.wikipedia.org/wiki/Talk:Carlos_Zeballos,2016-05-26T19:19:15.249Z,https://en.wikipedia.org/w/index.php?diff=722229004&oldid=720456554,Cyberbot II,http://en.wikipedia.org/wiki/User:Cyberbot II,en,http://en.wikipedia.org
#de.wikipedia,/* Gesang */ ...auch Amanda war nur auf der einer Sonderedition des Albums beiliegenden Bonus-Live-CD vertreten.,-7,,"List(null, null, null, null, null, null, null)",false,false,false,false,article,Avantasia,http://de.wikipedia.org/wiki/Avantasia,2016-05-26T19:19:15.309Z,https://de.wikipedia.org/w/index.php?diff=154727294&oldid=154727191,Johnny.m76,http://de.wikipedia.org/wiki/User:Johnny.m76,de,http://de.wikipedia.org
#es.wikipedia,/* Cuartos de final */,12,,"List(null, null, null, null, null, null, null)",false,false,false,false,article,Playoffs ACB 2015-16,http://es.wikipedia.org/wiki/Playoffs_ACB_2015-16,2016-05-26T19:19:15.369Z,https://es.wikipedia.org/w/index.php?diff=91308998&oldid=91308693,Politges13,http://es.wikipedia.org/wiki/User:Politges13,es,http://es.wikipedia.org
#en.wikipedia,/* X-Men (2000) */,1,,"List(Jacksonville, United States, US, USA, 30.287799835205078, 30.287799835205078, FL)",true,false,false,false,article,X-Men (film series),http://en.wikipedia.org/wiki/X-Men_(film_series),2016-05-26T19:19:15.466Z,https://en.wikipedia.org/w/index.php?diff=722229000&oldid=722228933,2602:306:32A5:AD50:A576:B4DA:11C9:8128,http://en.wikipedia.org/wiki/User:2602:306:32A5:AD50:A576:B4DA:11C9:8128,en,http://en.wikipedia.org
#es.wikipedia,187.235.134.193 activó [[Especial:FiltroAntiAbusos/13|filtro 13]] al realizar la acción «edit» en [[02Gravity Falls10]]. Medidas adoptadas: Advertir ([[Especial:RegistroAbusos/5531680|detalles]]),null,hit,"List(null, Mexico, MX, MEX, 19.43000030517578, 19.43000030517578, null)",true,false,false,false,special,Especial:Log/abusefilter,http://es.wikipedia.org/wiki/Especial:Log/abusefilter,2016-05-26T19:19:15.941Z,,187.235.134.193,http://es.wikipedia.org/wiki/User:187.235.134.193,es,http://es.wikipedia.org
#es.wikipedia,187.235.134.193 activó [[Especial:FiltroAntiAbusos/84|filtro 84]] al realizar la acción «edit» en [[02Gravity Falls10]]. Medidas adoptadas: Etiquetar ([[Especial:RegistroAbusos/55316

Notice the **geocoding** column has embedded data.

You can expand the fields by clicking the right triangle in each row.

But we can also reference the sub-fields directly as we see in the following SQL statement:

In [0]:
%sql

SELECT channel, page, geocoding.city, geocoding.latitude, geocoding.longitude 
FROM wiki_edits 
WHERE geocoding.city IS NOT NULL

channel,page,city,latitude,longitude
#en.wikipedia,X-Men (film series),Jacksonville,30.287799835205078,30.287799835205078
#it.wikipedia,Romano d'Ezzelino,Verona,45.429901123046875,45.429901123046875
#en.wikipedia,Lisa Goes Gaga,Saint Charles,41.93519973754883,41.93519973754883
#fr.wikipedia,Giessen (Allemagne),Brain-sur-Allonnes,47.293399810791016,47.293399810791016
#en.wikipedia,County Upper School,Milton Keynes,52.03860092163086,52.03860092163086
#it.wikipedia,Kia Sephia,Corea Castelluccia San Paolo,41.88330078125,41.88330078125
#en.wikipedia,Special:Log/abusefilter,Hamilton,43.251399993896484,43.251399993896484
#en.wikipedia,Virgil (wrestler),Hamilton,43.251399993896484,43.251399993896484
#it.wikipedia,Gran Premio d'Europa 2007,Milan,45.46670150756836,45.46670150756836
#nl.wikipedia,Geel (stad),Birmingham,52.47679901123047,52.47679901123047


### Review: Reading from JSON w/ InferSchema

While there are similarities between reading in CSV & JSON there are some key differences:
* We only need one job even when inferring the schema.
* There is no header which is why there isn't a second job in this case - the column names are extracted from the JSON object's attributes.
* Unlike CSV which reads in 100% of the data, the JSON reader only samples the data.  
**Note:** In Spark 2.2 the behavior was changed to read in the entire JSON file.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Reading from JSON w/ User-Defined Schema

To avoid the extra job, we can (just like we did with CSV) specify the schema for the `DataFrame`.

### Step #1 - Create the Schema

Compared to our CSV example, the structure of this data is a little more complex.

Note that we can support complex data types as seen in the field `geocoding`.

In [0]:
# Required for StructField, StringType, IntegerType, etc.
from pyspark.sql.types import *

jsonSchema = StructType([
  StructField("channel", StringType(), True),
  StructField("comment", StringType(), True),
  StructField("delta", IntegerType(), True),
  StructField("flag", StringType(), True),
  StructField("geocoding", StructType([
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("countryCode2", StringType(), True),
    StructField("countryCode3", StringType(), True),
    StructField("stateProvince", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
  ]), True),
  StructField("isAnonymous", BooleanType(), True),
  StructField("isNewPage", BooleanType(), True),
  StructField("isRobot", BooleanType(), True),
  StructField("isUnpatrolled", BooleanType(), True),
  StructField("namespace", StringType(), True),
  StructField("page", StringType(), True),
  StructField("pageURL", StringType(), True),
  StructField("timestamp", StringType(), True),
  StructField("url", StringType(), True),
  StructField("user", StringType(), True),
  StructField("userURL", StringType(), True),
  StructField("wikipediaURL", StringType(), True),
  StructField("wikipedia", StringType(), True)
])

That was a lot of typing to get our schema!

For a small file, manually creating the the schema may not be worth the effort.

However, for a large file, the time to manually create the schema may be worth the trade off of a really long infer-schema process.

### Step #2 - Read in the JSON

Next, we will read in the JSON file and once again print its schema.

In [0]:
(spark.read            # The DataFrameReader
  .schema(jsonSchema)  # Use the specified schema
  .json(jsonFile)      # Creates a DataFrame from JSON after reading in the file
  .printSchema()
)

root
-- channel: string (nullable = true)
-- comment: string (nullable = true)
-- delta: integer (nullable = true)
-- flag: string (nullable = true)
-- geocoding: struct (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- countryCode2: string (nullable = true)
 |-- countryCode3: string (nullable = true)
 |-- stateProvince: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
-- isAnonymous: boolean (nullable = true)
-- isNewPage: boolean (nullable = true)
-- isRobot: boolean (nullable = true)
-- isUnpatrolled: boolean (nullable = true)
-- namespace: string (nullable = true)
-- page: string (nullable = true)
-- pageURL: string (nullable = true)
-- timestamp: string (nullable = true)
-- url: string (nullable = true)
-- user: string (nullable = true)
-- userURL: string (nullable = true)
-- wikipediaURL: string (nullable = true)
-- wikipedia: string (nullable = true)

### Review: Reading from JSON w/ User-Defined Schema
* Just like CSV, providing the schema avoids the extra jobs.
* The schema allows us to rename columns and specify alternate data types.
* Can get arbitrarily complex in its structure.

Let's take a look at some of the other details of the `DataFrame` we just created for comparison sake.

In [0]:
jsonDF = (spark.read
  .schema(jsonSchema)
  .json(jsonFile)    
)
print("Partitions: " + str(jsonDF.rdd.getNumPartitions()))
printRecordsPerPartition(jsonDF)
print("-"*80)

Partitions: 2
Per-Partition Counts
#1: 5,961
#2: 39
--------------------------------------------------------------------------------

And of course we can view that data here:

In [0]:
display(jsonDF)

channel,comment,delta,flag,geocoding,isAnonymous,isNewPage,isRobot,isUnpatrolled,namespace,page,pageURL,timestamp,url,user,userURL,wikipediaURL,wikipedia
#pl.wikipedia,"plik, drobne techniczne, kat.",116,,"List(null, null, null, null, null, null, null)",false,false,false,false,article,Banda bat´ki Knysza,http://pl.wikipedia.org/wiki/Banda_bat´ki_Knysza,2016-05-26T19:19:14.367Z,https://pl.wikipedia.org/w/index.php?diff=45897271&oldid=42573382,Czymjestlogika,http://pl.wikipedia.org/wiki/User:Czymjestlogika,http://pl.wikipedia.org,pl
#fr.wikipedia,/* Résumé */,-1,!M,"List(null, null, null, null, null, null, null)",false,false,false,true,article,La Méprise (roman),http://fr.wikipedia.org/wiki/La_Méprise_(roman),2016-05-26T19:19:14.515Z,https://fr.wikipedia.org/w/index.php?diff=126531898&oldid=118306121&rcid=188792265,Polipille,http://fr.wikipedia.org/wiki/User:Polipille,http://fr.wikipedia.org,fr
#en.wikipedia,Rescuing 1 sources. #IABot,110,B,"List(null, null, null, null, null, null, null)",false,false,true,false,article,Carlos Zeballos,http://en.wikipedia.org/wiki/Carlos_Zeballos,2016-05-26T19:19:14.586Z,https://en.wikipedia.org/w/index.php?diff=722229003&oldid=720692086,Cyberbot II,http://en.wikipedia.org/wiki/User:Cyberbot II,http://en.wikipedia.org,en
#sv.wikipedia,Botskapande Grekland omdirigering,26,NB,"List(null, null, null, null, null, null, null)",false,true,true,false,article,Doliana,http://sv.wikipedia.org/wiki/Doliana,2016-05-26T19:19:14.739Z,https://sv.wikipedia.org/w/index.php?oldid=35100063&rcid=81393046,Lsjbot,http://sv.wikipedia.org/wiki/User:Lsjbot,http://sv.wikipedia.org,sv
#en.wikipedia,Notification of altered sources needing review #IABot,1063,B,"List(null, null, null, null, null, null, null)",false,false,true,false,talk,Talk:Carlos Zeballos,http://en.wikipedia.org/wiki/Talk:Carlos_Zeballos,2016-05-26T19:19:15.249Z,https://en.wikipedia.org/w/index.php?diff=722229004&oldid=720456554,Cyberbot II,http://en.wikipedia.org/wiki/User:Cyberbot II,http://en.wikipedia.org,en
#de.wikipedia,/* Gesang */ ...auch Amanda war nur auf der einer Sonderedition des Albums beiliegenden Bonus-Live-CD vertreten.,-7,,"List(null, null, null, null, null, null, null)",false,false,false,false,article,Avantasia,http://de.wikipedia.org/wiki/Avantasia,2016-05-26T19:19:15.309Z,https://de.wikipedia.org/w/index.php?diff=154727294&oldid=154727191,Johnny.m76,http://de.wikipedia.org/wiki/User:Johnny.m76,http://de.wikipedia.org,de
#es.wikipedia,/* Cuartos de final */,12,,"List(null, null, null, null, null, null, null)",false,false,false,false,article,Playoffs ACB 2015-16,http://es.wikipedia.org/wiki/Playoffs_ACB_2015-16,2016-05-26T19:19:15.369Z,https://es.wikipedia.org/w/index.php?diff=91308998&oldid=91308693,Politges13,http://es.wikipedia.org/wiki/User:Politges13,http://es.wikipedia.org,es
#en.wikipedia,/* X-Men (2000) */,1,,"List(Jacksonville, United States, US, USA, FL, 30.287799835205078, 30.287799835205078)",true,false,false,false,article,X-Men (film series),http://en.wikipedia.org/wiki/X-Men_(film_series),2016-05-26T19:19:15.466Z,https://en.wikipedia.org/w/index.php?diff=722229000&oldid=722228933,2602:306:32A5:AD50:A576:B4DA:11C9:8128,http://en.wikipedia.org/wiki/User:2602:306:32A5:AD50:A576:B4DA:11C9:8128,http://en.wikipedia.org,en
#es.wikipedia,187.235.134.193 activó [[Especial:FiltroAntiAbusos/13|filtro 13]] al realizar la acción «edit» en [[02Gravity Falls10]]. Medidas adoptadas: Advertir ([[Especial:RegistroAbusos/5531680|detalles]]),null,hit,"List(null, Mexico, MX, MEX, null, 19.43000030517578, 19.43000030517578)",true,false,false,false,special,Especial:Log/abusefilter,http://es.wikipedia.org/wiki/Especial:Log/abusefilter,2016-05-26T19:19:15.941Z,,187.235.134.193,http://es.wikipedia.org/wiki/User:187.235.134.193,http://es.wikipedia.org,es
#es.wikipedia,187.235.134.193 activó [[Especial:FiltroAntiAbusos/84|filtro 84]] al realizar la acción «edit» en [[02Gravity Falls10]]. Medidas adoptadas: Etiquetar ([[Especial:RegistroAbusos/55316

## Next steps

Start the next lesson, [Reading Data - Parquet]($./3.Reading%20Data%20-%20Parquet)